In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
listings = pd.read_csv('../data/london_listings.csv')

In [3]:
#listings.info()

In [4]:
df_clean = listings[['id', 'price', 'host_is_superhost', 'latitude', 'longitude', 'neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'instant_bookable', 'host_has_profile_pic', 'host_identity_verified', 'number_of_reviews']]

In [5]:
#df_clean

In [6]:
df_clean = df_clean[df_clean['price'] != 0.0]

In [7]:
#reset index to listing id
df_clean = df_clean.set_index('id')
#df_clean

In [8]:
df_clean[df_clean['price'] == 0.0]

,price,host_is_superhost,latitude,longitude,neighbourhood_cleansed,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,instant_bookable,host_has_profile_pic,host_identity_verified,number_of_reviews
id,,,,,,,,,,,,,,,


In [9]:
#make price a float
df_clean['price'] = df_clean['price'].str.replace(',', '')
df_clean['price'] = df_clean['price'].str.replace('$', '')
df_clean['price'] = df_clean['price'].astype(float)
df_clean

,price,host_is_superhost,latitude,longitude,neighbourhood_cleansed,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,instant_bookable,host_has_profile_pic,host_identity_verified,number_of_reviews
id,,,,,,,,,,,,,,,
11551,97.0,f,51.46225,-0.11732,Lambeth,Entire home/apt,4,1 bath,1.0,3.0,"[""Cable TV"", ""Coffee maker"", ""Carbon monoxide ...",t,t,t,192
13913,40.0,f,51.56802,-0.11121,Islington,Private room,2,1 shared bath,1.0,0.0,"[""Cable TV"", ""Bed linens"", ""Coffee maker"", ""Pa...",f,t,t,21
15400,75.0,f,51.48796,-0.16898,Kensington and Chelsea,Entire home/apt,2,1 bath,1.0,1.0,"[""Cable TV"", ""Coffee maker"", ""Dryer"", ""Dishes ...",t,t,t,89
17402,307.0,t,51.52195,-0.14094,Westminster,Entire home/apt,6,2 baths,3.0,3.0,"[""Bed linens"", ""Coffee maker"", ""Dryer"", ""Paid ...",f,t,t,42
17506,150.0,f,51.47935,-0.19743,Hammersmith and Fulham,Private room,2,1 private bath,1.0,1.0,"[""TV"", ""Heating"", ""Shower gel"", ""Wifi"", ""Smoke...",f,t,t,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46259322,90.0,t,51.51343,-0.13286,Westminster,Entire home/apt,2,1 bath,1.0,1.0,"[""TV"", ""Fire extinguisher"", ""Heating"", ""Dryer""...",f,t,t,0
46259459,87.0,f,51.50755,-0.15069,Westminster,Private room,2,2 shared baths,1.0,2.0,"[""Dryer"", ""Private entrance"", ""First aid kit"",...",t,t,f,0
46259503,26.0,f,51.54920,0.07109,Redbridge,Private room,2,1.5 shared baths,1.0,1.0,"[""Fire extinguisher"", ""Body soap"", ""Heating"", ...",t,t,t,0


In [10]:
df_clean['price'].describe()

count    76984.000000
mean       112.107932
std        325.929453
min          0.000000
25%         43.000000
50%         75.000000
75%        121.000000
max      18673.000000
Name: price, dtype: float64

In [11]:
#make host_is_superhost a integer and make any nulls equal to zero(non-superhost)
df_clean['host_is_superhost'] = (df_clean['host_is_superhost'] == 't').astype(int)
df_clean['host_is_superhost'] = df_clean['host_is_superhost'].fillna(0)
df_clean[['host_is_superhost']]

,host_is_superhost
id,
11551,0
13913,0
15400,0
17402,1
17506,0
...,...
46259322,1
46259459,0
46259503,0


In [12]:
#make neighborhoods into dummy variables
df_clean = pd.get_dummies(df_clean, columns=['neighbourhood_cleansed'])
df_clean.head()

,price,host_is_superhost,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,...,neighbourhood_cleansed_Merton,neighbourhood_cleansed_Newham,neighbourhood_cleansed_Redbridge,neighbourhood_cleansed_Richmond upon Thames,neighbourhood_cleansed_Southwark,neighbourhood_cleansed_Sutton,neighbourhood_cleansed_Tower Hamlets,neighbourhood_cleansed_Waltham Forest,neighbourhood_cleansed_Wandsworth,neighbourhood_cleansed_Westminster
id,,,,,,,,,,,,,,,,,,,,,
11551,97.0,0,51.46225,-0.11732,Entire home/apt,4,1 bath,1.0,3.0,"[""Cable TV"", ""Coffee maker"", ""Carbon monoxide ...",...,0,0,0,0,0,0,0,0,0,0
13913,40.0,0,51.56802,-0.11121,Private room,2,1 shared bath,1.0,0.0,"[""Cable TV"", ""Bed linens"", ""Coffee maker"", ""Pa...",...,0,0,0,0,0,0,0,0,0,0
15400,75.0,0,51.48796,-0.16898,Entire home/apt,2,1 bath,1.0,1.0,"[""Cable TV"", ""Coffee maker"", ""Dryer"", ""Dishes ...",...,0,0,0,0,0,0,0,0,0,0
17402,307.0,1,51.52195,-0.14094,Entire home/apt,6,2 baths,3.0,3.0,"[""Bed linens"", ""Coffee maker"", ""Dryer"", ""Paid ...",...,0,0,0,0,0,0,0,0,0,1
17506,150.0,0,51.47935,-0.19743,Private room,2,1 private bath,1.0,1.0,"[""TV"", ""Heating"", ""Shower gel"", ""Wifi"", ""Smoke...",...,0,0,0,0,0,0,0,0,0,0


In [13]:
#make room_type into dummy variables
df_clean = pd.get_dummies(df_clean, columns=['room_type'])
df_clean.head()

,price,host_is_superhost,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,amenities,instant_bookable,...,neighbourhood_cleansed_Southwark,neighbourhood_cleansed_Sutton,neighbourhood_cleansed_Tower Hamlets,neighbourhood_cleansed_Waltham Forest,neighbourhood_cleansed_Wandsworth,neighbourhood_cleansed_Westminster,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
id,,,,,,,,,,,,,,,,,,,,,
11551,97.0,0,51.46225,-0.11732,4,1 bath,1.0,3.0,"[""Cable TV"", ""Coffee maker"", ""Carbon monoxide ...",t,...,0,0,0,0,0,0,1,0,0,0
13913,40.0,0,51.56802,-0.11121,2,1 shared bath,1.0,0.0,"[""Cable TV"", ""Bed linens"", ""Coffee maker"", ""Pa...",f,...,0,0,0,0,0,0,0,0,1,0
15400,75.0,0,51.48796,-0.16898,2,1 bath,1.0,1.0,"[""Cable TV"", ""Coffee maker"", ""Dryer"", ""Dishes ...",t,...,0,0,0,0,0,0,1,0,0,0
17402,307.0,1,51.52195,-0.14094,6,2 baths,3.0,3.0,"[""Bed linens"", ""Coffee maker"", ""Dryer"", ""Paid ...",f,...,0,0,0,0,0,1,1,0,0,0
17506,150.0,0,51.47935,-0.19743,2,1 private bath,1.0,1.0,"[""TV"", ""Heating"", ""Shower gel"", ""Wifi"", ""Smoke...",f,...,0,0,0,0,0,0,0,0,1,0


In [14]:
#remove null values from bathrooms_text column and change from categorical to numerical
df_clean['bathrooms_text'].nunique()
df_clean['bathrooms_text'].unique()
df_clean['bathrooms_text'].isnull().sum()
df_clean[df_clean['bathrooms_text'].isnull()]
df_clean['bathrooms_text'].fillna(2, inplace=True)
clean_bathrooms_text = {"1 bath": 1,
                        "1 shared bath": 1,
                        "2 baths": 2,
                        "1 private bath": 1,
                        "1.5 shared baths": 1.5,
                        "1.5 baths": 1.5,
                        "0 shared baths": 0,
                        "2.5 shared baths": 2.5,
                        "Shared half-bath": 0.5,
                        "4 baths": 4,
                        "3 baths": 3,
                        "0 baths": 0,
                        "3 shared baths": 3,
                        "3.5 baths": 3.5,
                        "Half-bath": 0.5,
                        "5 baths": 5,
                        "4.5 baths": 4.5,
                        "5 shared baths": 5,
                        "3.5 shared baths": 3.5,
                        "Private half-bath": 0.5,
                        "7 baths": 7,
                        "4 shared baths": 4,
                        "6 baths": 6,
                        "6 shared baths": 6,
                        "5.5 baths": 5.5,
                        "10 baths": 10,
                        "8.5 baths": 8.5,
                        "7 shared baths": 7,
                        "4.5 shared baths": 4.5,
                        "6.5 baths": 6.5,
                        "8 shared baths": 8,
                        "17 baths": 17,
                        "11 baths": 11,
                        "7.5 baths": 7.5,
                        "8 baths": 8,
                        "10.5 baths": 10.5,
                        "9 baths": 9,
                        "12 baths": 12,
                        "9 shared baths": 9,
                        "35 baths": 35,
                        "2 shared baths": 2,
                        "2.5 baths": 2.5}
df_clean['bathrooms_text'] = df_clean['bathrooms_text'].replace(clean_bathrooms_text)
df_clean[['bathrooms_text']]

,bathrooms_text
id,
11551,1.0
13913,1.0
15400,1.0
17402,2.0
17506,1.0
...,...
46259322,1.0
46259459,2.0
46259503,1.5


In [15]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76984 entries, 11551 to 46261520
Data columns (total 50 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   price                                          76984 non-null  float64
 1   host_is_superhost                              76984 non-null  int64  
 2   latitude                                       76984 non-null  float64
 3   longitude                                      76984 non-null  float64
 4   accommodates                                   76984 non-null  int64  
 5   bathrooms_text                                 76984 non-null  float64
 6   bedrooms                                       72238 non-null  float64
 7   beds                                           75904 non-null  float64
 8   amenities                                      76984 non-null  object 
 9   instant_bookable                           

In [16]:
df_clean.dropna(subset=['bedrooms','beds'], how='all', inplace=True) #drop columns where bedrooms and beds both are null
#df_clean['bedrooms'].unique()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76882 entries, 11551 to 46261520
Data columns (total 50 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   price                                          76882 non-null  float64
 1   host_is_superhost                              76882 non-null  int64  
 2   latitude                                       76882 non-null  float64
 3   longitude                                      76882 non-null  float64
 4   accommodates                                   76882 non-null  int64  
 5   bathrooms_text                                 76882 non-null  float64
 6   bedrooms                                       72238 non-null  float64
 7   beds                                           75904 non-null  float64
 8   amenities                                      76882 non-null  object 
 9   instant_bookable                           

In [17]:
#group by beds and then find mean bedrooms for each number of beds
bedrooms_no_nulls = df_clean[df_clean['bedrooms'].notnull()]
mean_beds_rounded = bedrooms_no_nulls.groupby(by=['beds']).mean().round(0).reset_index()
mean_beds_rounded[['beds', 'bedrooms']]

,beds,bedrooms
0,0.0,1.0
1,1.0,1.0
2,2.0,2.0
3,3.0,2.0
4,4.0,3.0
5,5.0,4.0
6,6.0,4.0
7,7.0,4.0
8,8.0,4.0
9,9.0,5.0


In [18]:
#make prediction for null values in 'bedrooms' feature based on mean of bedrooms associated with beds column
fill_dict1 = mean_beds_rounded.set_index('beds')['bedrooms'].to_dict()
df_clean['bedrooms'] = df_clean['bedrooms'].replace(np.nan, df_clean['beds'].map(fill_dict1))
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76882 entries, 11551 to 46261520
Data columns (total 50 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   price                                          76882 non-null  float64
 1   host_is_superhost                              76882 non-null  int64  
 2   latitude                                       76882 non-null  float64
 3   longitude                                      76882 non-null  float64
 4   accommodates                                   76882 non-null  int64  
 5   bathrooms_text                                 76882 non-null  float64
 6   bedrooms                                       76882 non-null  float64
 7   beds                                           75904 non-null  float64
 8   amenities                                      76882 non-null  object 
 9   instant_bookable                           

In [19]:
#group by bedrooms and then find mean beds for each number of bedrooms
beds_no_nulls = df_clean[df_clean['beds'].notnull()]
mean_bedrooms_rounded = beds_no_nulls.groupby(by=['bedrooms']).mean().round(0).reset_index()
mean_bedrooms_rounded[['beds', 'bedrooms']]

,beds,bedrooms
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,6.0,5.0
5,7.0,6.0
6,9.0,7.0
7,13.0,8.0
8,6.0,9.0
9,9.0,10.0


In [20]:
#make prediction for null values in 'beds' feature based on mean of beds associated with bedrooms column
fill_dict2 = mean_bedrooms_rounded.set_index('bedrooms')['beds'].to_dict()
df_clean['beds'] = df_clean['beds'].replace(np.nan, df_clean['bedrooms'].map(fill_dict2))
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76882 entries, 11551 to 46261520
Data columns (total 50 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   price                                          76882 non-null  float64
 1   host_is_superhost                              76882 non-null  int64  
 2   latitude                                       76882 non-null  float64
 3   longitude                                      76882 non-null  float64
 4   accommodates                                   76882 non-null  int64  
 5   bathrooms_text                                 76882 non-null  float64
 6   bedrooms                                       76882 non-null  float64
 7   beds                                           76882 non-null  float64
 8   amenities                                      76882 non-null  object 
 9   instant_bookable                           

In [21]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76882 entries, 11551 to 46261520
Data columns (total 50 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   price                                          76882 non-null  float64
 1   host_is_superhost                              76882 non-null  int64  
 2   latitude                                       76882 non-null  float64
 3   longitude                                      76882 non-null  float64
 4   accommodates                                   76882 non-null  int64  
 5   bathrooms_text                                 76882 non-null  float64
 6   bedrooms                                       76882 non-null  float64
 7   beds                                           76882 non-null  float64
 8   amenities                                      76882 non-null  object 
 9   instant_bookable                           

In [22]:
df_clean_copy = df_clean.copy()

In [23]:
df_clean_copy[['amenities']]

,amenities
id,
11551,"[""Cable TV"", ""Coffee maker"", ""Carbon monoxide ..."
13913,"[""Cable TV"", ""Bed linens"", ""Coffee maker"", ""Pa..."
15400,"[""Cable TV"", ""Coffee maker"", ""Dryer"", ""Dishes ..."
17402,"[""Bed linens"", ""Coffee maker"", ""Dryer"", ""Paid ..."
17506,"[""TV"", ""Heating"", ""Shower gel"", ""Wifi"", ""Smoke..."
...,...
46259322,"[""TV"", ""Fire extinguisher"", ""Heating"", ""Dryer""..."
46259459,"[""Dryer"", ""Private entrance"", ""First aid kit"",..."
46259503,"[""Fire extinguisher"", ""Body soap"", ""Heating"", ..."


In [24]:
df_clean_copy['types'] = df_clean_copy['amenities'].apply(lambda x: type(x))

In [25]:
df_clean_copy['types'].unique()

array([<class 'str'>], dtype=object)

In [26]:
df_clean_copy['amenities'] = df_clean_copy['amenities'].apply(lambda x: x.strip("][").split(', ') if "[" in x else [x])

In [27]:
df_clean_copy['amenities']

id
11551       ["Cable TV", "Coffee maker", "Carbon monoxide ...
13913       ["Cable TV", "Bed linens", "Coffee maker", "Pa...
15400       ["Cable TV", "Coffee maker", "Dryer", "Dishes ...
17402       ["Bed linens", "Coffee maker", "Dryer", "Paid ...
17506       ["TV", "Heating", "Shower gel", "Wifi", "Smoke...
                                  ...                        
46259322    ["TV", "Fire extinguisher", "Heating", "Dryer"...
46259459    ["Dryer", "Private entrance", "First aid kit",...
46259503    ["Fire extinguisher", "Body soap", "Heating", ...
46259530    ["Fire extinguisher", "Heating", "Wifi", "Kitc...
46261520    ["Heating", "Wifi", "Kitchen", "Washer", "Esse...
Name: amenities, Length: 76882, dtype: object

In [28]:
s = df_clean_copy['amenities']

mlb = MultiLabelBinarizer()

amenity_dummies = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=df_clean.index)

In [29]:
amenity_dummies['price'] = df_clean_copy['price']

In [30]:
amenity_price_corr = amenity_dummies.corr()

In [31]:
ascending_price = amenity_price_corr.sort_values(by=['price'], ascending=False)

In [32]:
ascending_price.head(19)

,,""" Miele electric stove""","""14\"" TV with Netflix","""17\"" TV""","""20\"" HDTV with standard cable""","""24\"" HDTV with Netflix","""32\"" HDTV with Chromecast","""32\"" HDTV with Netflix""","""32\"" HDTV with standard cable""","""32\"" HDTV""",...,"The Platinum Range body soap""","The Platinum Range conditioner""","The Platinum Range shampoo""","bespoke,hypoallergenic body soap""",premium cable,"premium cable""",standard cable,"standard cable""","works well for body :) body soap""",price
price,0.003520,0.000640,-0.000942,-0.000355,-0.000964,-0.000698,-0.000267,-0.000577,-0.000433,-0.000711,...,-0.000355,-0.000355,-0.000355,-0.000400,-0.000948,0.008919,0.001038,0.000437,-0.000687,1.000000
"""Air conditioning""",-0.020443,-0.000988,-0.000988,-0.000988,-0.000988,-0.000988,-0.000988,-0.000988,-0.000988,-0.001711,...,0.013169,0.013169,0.013169,-0.000988,-0.001711,0.003872,-0.001397,-0.003826,-0.000988,0.087795
"""TV""",-0.110197,-0.005324,-0.005324,-0.005324,-0.005324,-0.005324,-0.005324,-0.005324,-0.005324,-0.009222,...,0.002443,0.002443,0.002443,0.002443,-0.009222,-0.019923,-0.007530,-0.020623,0.002443,0.084993
"""Dishwasher""",-0.042055,0.006401,-0.002032,-0.002032,-0.002032,0.006401,-0.002032,-0.002032,0.006401,0.001350,...,0.006401,0.006401,0.006401,0.006401,0.011088,0.017192,0.009053,0.009551,0.006401,0.078707
"""Ironing board""",-0.001291,-0.000062,-0.000062,-0.000062,-0.000062,-0.000062,-0.000062,-0.000062,-0.000062,-0.000108,...,-0.000062,-0.000062,-0.000062,-0.000062,-0.000108,-0.000233,-0.000088,-0.000242,-0.000062,0.064458
"""Security system""",-0.000712,-0.000034,-0.000034,-0.000034,-0.000034,-0.000034,-0.000034,-0.000034,-0.000034,-0.000060,...,-0.000034,-0.000034,-0.000034,-0.000034,-0.000060,-0.000129,-0.000049,-0.000133,-0.000034,0.061466
"""Dryer""",-0.066496,-0.003213,-0.003213,-0.003213,-0.003213,-0.003213,-0.003213,0.004048,0.004048,0.002820,...,0.004048,0.004048,0.004048,0.004048,-0.005565,-0.000377,0.000591,-0.004944,0.004048,0.060623
"""Chef's kitchen""",-0.001319,-0.000064,-0.000064,-0.000064,-0.000064,-0.000064,-0.000064,-0.000064,-0.000064,-0.000110,...,-0.000064,-0.000064,-0.000064,-0.000064,-0.000110,-0.000238,-0.000090,-0.000247,-0.000064,0.058543
"""Coffee maker""",-0.039656,0.006789,-0.001916,0.006789,-0.001916,-0.001916,0.006789,-0.001916,-0.001916,0.006733,...,-0.001916,-0.001916,-0.001916,0.006789,0.001707,0.006790,-0.002710,0.006065,0.006789,0.051785
"""Balcony""",-0.000659,-0.000032,-0.000032,-0.000032,-0.000032,-0.000032,-0.000032,-0.000032,-0.000032,-0.000055,...,-0.000032,-0.000032,-0.000032,-0.000032,-0.000055,-0.000119,-0.000045,-0.000123,-0.000032,0.051044


In [33]:
df_corr = df_clean.corr()

In [34]:
df_corr.sort_values(by=['price'], ascending=False)

,price,host_is_superhost,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,number_of_reviews,neighbourhood_cleansed_Barking and Dagenham,...,neighbourhood_cleansed_Southwark,neighbourhood_cleansed_Sutton,neighbourhood_cleansed_Tower Hamlets,neighbourhood_cleansed_Waltham Forest,neighbourhood_cleansed_Wandsworth,neighbourhood_cleansed_Westminster,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
price,1.000000,-0.016382,-0.006274,-0.030609,0.190364,0.143808,0.172718,0.155744,-0.027452,-0.010188,...,-0.007202,-0.004082,-0.022699,-0.017669,-0.003395,0.078047,0.145222,0.004127,-0.143944,-0.013877
accommodates,0.190364,-0.013006,-0.009601,-0.046104,1.000000,0.486262,0.787554,0.797590,-0.028792,-0.017893,...,-0.005902,-0.017359,-0.034746,-0.010294,0.022220,0.077723,0.541584,-0.006733,-0.536947,-0.029808
bedrooms,0.172718,-0.026788,-0.023869,-0.055924,0.787554,0.598054,1.000000,0.771694,-0.063025,-0.012749,...,-0.013433,-0.007357,-0.048150,0.002453,0.047003,0.026228,0.415666,-0.015851,-0.405520,-0.049127
beds,0.155744,-0.011386,-0.009595,-0.042877,0.797590,0.495897,0.771694,1.000000,-0.029980,-0.007055,...,-0.011715,-0.005753,-0.038474,0.000750,0.022576,0.039373,0.384789,0.005216,-0.391048,0.022159
room_type_Entire home/apt,0.145222,-0.032898,-0.006950,-0.065785,0.541584,0.164959,0.415666,0.384789,-0.076649,-0.036177,...,-0.005607,-0.027818,-0.047913,-0.028275,0.003943,0.129802,1.000000,-0.095297,-0.969083,-0.099415
bathrooms_text,0.143808,-0.016087,-0.015394,-0.048171,0.486262,1.000000,0.598054,0.495897,-0.038343,-0.007970,...,-0.021878,-0.008643,-0.027442,-0.010379,0.031006,0.044986,0.164959,0.012613,-0.167063,-0.003661
neighbourhood_cleansed_Westminster,0.078047,-0.006744,0.026772,-0.125124,0.077723,0.044986,0.026228,0.039373,0.012292,-0.025696,...,-0.086467,-0.022230,-0.110460,-0.047716,-0.082282,1.000000,0.129802,0.008388,-0.130698,-0.005613
neighbourhood_cleansed_Kensington and Chelsea,0.050934,0.010121,-0.063557,-0.174900,0.056471,0.037561,0.033191,0.037336,0.001152,-0.019788,...,-0.066588,-0.017119,-0.085064,-0.036745,-0.063364,-0.096497,0.122500,0.074331,-0.136839,0.005752
neighbourhood_cleansed_City of London,0.043457,-0.000796,0.007949,0.028373,0.005485,-0.010908,-0.014674,-0.007575,0.001104,-0.005539,...,-0.018638,-0.004792,-0.023809,-0.010285,-0.017736,-0.027009,0.049964,0.001388,-0.049830,-0.003054
neighbourhood_cleansed_Islington,0.011292,-0.005560,0.181385,0.053401,-0.008088,-0.027123,-0.018728,-0.018352,0.012955,-0.018066,...,-0.060790,-0.015628,-0.077659,-0.033546,-0.057848,-0.088096,0.032182,-0.002781,-0.029998,-0.010056


In [35]:
#change instant_bookable from categorical data to numerical 
df_clean['instant_bookable'] = (df_clean['instant_bookable'] == 't').astype(int)
df_clean.head()

,price,host_is_superhost,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,amenities,instant_bookable,...,neighbourhood_cleansed_Southwark,neighbourhood_cleansed_Sutton,neighbourhood_cleansed_Tower Hamlets,neighbourhood_cleansed_Waltham Forest,neighbourhood_cleansed_Wandsworth,neighbourhood_cleansed_Westminster,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
id,,,,,,,,,,,,,,,,,,,,,
11551,97.0,0,51.46225,-0.11732,4,1.0,1.0,3.0,"[""Cable TV"", ""Coffee maker"", ""Carbon monoxide ...",1,...,0,0,0,0,0,0,1,0,0,0
13913,40.0,0,51.56802,-0.11121,2,1.0,1.0,0.0,"[""Cable TV"", ""Bed linens"", ""Coffee maker"", ""Pa...",0,...,0,0,0,0,0,0,0,0,1,0
15400,75.0,0,51.48796,-0.16898,2,1.0,1.0,1.0,"[""Cable TV"", ""Coffee maker"", ""Dryer"", ""Dishes ...",1,...,0,0,0,0,0,0,1,0,0,0
17402,307.0,1,51.52195,-0.14094,6,2.0,3.0,3.0,"[""Bed linens"", ""Coffee maker"", ""Dryer"", ""Paid ...",0,...,0,0,0,0,0,1,1,0,0,0
17506,150.0,0,51.47935,-0.19743,2,1.0,1.0,1.0,"[""TV"", ""Heating"", ""Shower gel"", ""Wifi"", ""Smoke...",0,...,0,0,0,0,0,0,0,0,1,0


In [36]:
#change host_has_profile_pic from categorical data to numerical and fill missing values with 0 (no profile pic)
df_clean['host_has_profile_pic'] = (df_clean['host_has_profile_pic'] == 't').astype(int)
df_clean['host_has_profile_pic'] = df_clean['host_has_profile_pic'].fillna(0)
df_clean[['host_has_profile_pic']]

,host_has_profile_pic
id,
11551,1
13913,1
15400,1
17402,1
17506,1
...,...
46259322,1
46259459,1
46259503,1


In [37]:
#change host_identity_verified from categorical data to numerical and fill missing values with 0 (not verified)
df_clean['host_identity_verified'] = (df_clean['host_identity_verified'] == 't').astype(int)
df_clean['host_identity_verified'] = df_clean['host_identity_verified'].fillna(0)
df_clean[['host_identity_verified']]

,host_identity_verified
id,
11551,1
13913,1
15400,1
17402,1
17506,1
...,...
46259322,1
46259459,0
46259503,1


In [38]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76882 entries, 11551 to 46261520
Data columns (total 50 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   price                                          76882 non-null  float64
 1   host_is_superhost                              76882 non-null  int64  
 2   latitude                                       76882 non-null  float64
 3   longitude                                      76882 non-null  float64
 4   accommodates                                   76882 non-null  int64  
 5   bathrooms_text                                 76882 non-null  float64
 6   bedrooms                                       76882 non-null  float64
 7   beds                                           76882 non-null  float64
 8   amenities                                      76882 non-null  object 
 9   instant_bookable                           

In [39]:
amenities = amenity_dummies[['"Air conditioning"', '"TV"', '"Dishwasher"', '"Ironing board"', '"Security system"', '"Dryer"', '"Chef\'s kitchen"', '"Coffee maker"', '"Balcony"', '"Terrace"', '"Espresso machine"', '"Crib"', '"Dining area for 8 people"', '"Indoor fireplace"', '"Private entrance"', '"Oven"', '"High chair"', '"Shampoo"' ]]

In [40]:
amenities.head()

,"""Air conditioning""","""TV""","""Dishwasher""","""Ironing board""","""Security system""","""Dryer""","""Chef's kitchen""","""Coffee maker""","""Balcony""","""Terrace""","""Espresso machine""","""Crib""","""Dining area for 8 people""","""Indoor fireplace""","""Private entrance""","""Oven""","""High chair""","""Shampoo"""
id,,,,,,,,,,,,,,,,,,
11551,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0
13913,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1
15400,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
17402,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1
17506,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [41]:
drop_amenities = df_clean.drop(['amenities'], axis=1)
drop_amenities

,price,host_is_superhost,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,instant_bookable,host_has_profile_pic,...,neighbourhood_cleansed_Southwark,neighbourhood_cleansed_Sutton,neighbourhood_cleansed_Tower Hamlets,neighbourhood_cleansed_Waltham Forest,neighbourhood_cleansed_Wandsworth,neighbourhood_cleansed_Westminster,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
id,,,,,,,,,,,,,,,,,,,,,
11551,97.0,0,51.46225,-0.11732,4,1.0,1.0,3.0,1,1,...,0,0,0,0,0,0,1,0,0,0
13913,40.0,0,51.56802,-0.11121,2,1.0,1.0,0.0,0,1,...,0,0,0,0,0,0,0,0,1,0
15400,75.0,0,51.48796,-0.16898,2,1.0,1.0,1.0,1,1,...,0,0,0,0,0,0,1,0,0,0
17402,307.0,1,51.52195,-0.14094,6,2.0,3.0,3.0,0,1,...,0,0,0,0,0,1,1,0,0,0
17506,150.0,0,51.47935,-0.19743,2,1.0,1.0,1.0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46259322,90.0,1,51.51343,-0.13286,2,1.0,1.0,1.0,0,1,...,0,0,0,0,0,1,1,0,0,0
46259459,87.0,0,51.50755,-0.15069,2,2.0,1.0,2.0,1,1,...,0,0,0,0,0,1,0,0,1,0
46259503,26.0,0,51.54920,0.07109,2,1.5,1.0,1.0,1,1,...,0,0,0,0,0,0,0,0,1,0


In [42]:
result = pd.concat([drop_amenities, amenities], axis=1)

In [43]:
result.head()

,price,host_is_superhost,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,instant_bookable,host_has_profile_pic,...,"""Balcony""","""Terrace""","""Espresso machine""","""Crib""","""Dining area for 8 people""","""Indoor fireplace""","""Private entrance""","""Oven""","""High chair""","""Shampoo"""
id,,,,,,,,,,,,,,,,,,,,,
11551,97.0,0,51.46225,-0.11732,4,1.0,1.0,3.0,1,1,...,0,0,0,1,0,0,0,1,1,0
13913,40.0,0,51.56802,-0.11121,2,1.0,1.0,0.0,0,1,...,0,0,0,1,0,0,0,1,0,1
15400,75.0,0,51.48796,-0.16898,2,1.0,1.0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,1
17402,307.0,1,51.52195,-0.14094,6,2.0,3.0,3.0,0,1,...,0,0,0,0,0,0,0,1,0,1
17506,150.0,0,51.47935,-0.19743,2,1.0,1.0,1.0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [44]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76882 entries, 11551 to 46261520
Data columns (total 67 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   price                                          76882 non-null  float64
 1   host_is_superhost                              76882 non-null  int64  
 2   latitude                                       76882 non-null  float64
 3   longitude                                      76882 non-null  float64
 4   accommodates                                   76882 non-null  int64  
 5   bathrooms_text                                 76882 non-null  float64
 6   bedrooms                                       76882 non-null  float64
 7   beds                                           76882 non-null  float64
 8   instant_bookable                               76882 non-null  int64  
 9   host_has_profile_pic                       